This is a notebook that calculates neutron capture cross sections at a given T9 in units of millibarns $(mb)$

Here we import the relavent python packages

In [ ]:
import sys
!{sys.executable} -m pip install --quiet wnutils

import sys
!{sys.executable} -m pip install --quiet astropy

import os, io, requests
from lxml import etree

import wnutils.xml as wx
import numpy as np
from astropy import constants as pc
from astropy import units as u

Now we download an xml that contains the reaction data

In [ ]:
my_xml = wx.Xml(io.BytesIO(requests.get('https://osf.io/vqhxr/download').content))

In [ ]:
t9 = 0.8
T = t9 * 1.e9 * u.K
Z = str(43)
A_min = str(95)
A_max = str(100)

In [ ]:
v_t = np.sqrt((2 * pc.k_B.decompose(u.cgs.bases) * T) / pc.m_n.decompose(u.cgs.bases))

In [ ]:
sig_dict = {}

filter = {}

In [ ]:
nuclides = {}
#nuc = my_xml.get_nuclide_data("[z = " + Z +"]")
nuc = my_xml.get_nuclide_data("[z = "+ Z +" and a >= "+ A_min +" and a <= "+ A_max +"]")
nuclides.update(nuc)

We now specify that we only want $(n,\gamma)$ reactions

In [ ]:
reac = my_xml.get_reaction_data("[reactant = 'n' and count(reactant) = 2  and product = 'gamma' and count(product) = 2]")

In [ ]:
for s in reac:
    reaction = reac[s]
    rate = reaction.compute_rate(t9) * u.cm**3 /u.second /u.mol 
    sigma = (rate / (pc.N_A * v_t)).to(u.barn)
        
    usigma = 1000 * sigma / u.barn
    sig_dict[s] = usigma 

In [ ]:
for reaction in reac:
    for reactant in reac[reaction].reactants:
        if reactant in nuclides:
            filter[reaction] = reaction

In [ ]:
for key in list(sig_dict.keys()):
    if key not in filter:
        del sig_dict[key]

In [ ]:
print('Cross Sections at T9 of ' + str(t9))

for k,v in sorted(sig_dict.items()):
    print(str(k)+': ' + str(v) + ' mb')